In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import os

# Set visualization style for plots
sns.set_style("whitegrid")

# Load Anonymized Data
print("Step 1: Loading anonymized data...")
DATA_DIR = '../data'
CALLS_PATH = os.path.join(DATA_DIR, 'call_created_anonymized.csv')
CENSUS_PATH = os.path.join(DATA_DIR, 'census_check_anonymized.csv')

try:
    calls_df = pd.read_csv(CALLS_PATH, parse_dates=['original_timestamp'], low_memory=False)
    census_df = pd.read_csv(CENSUS_PATH, parse_dates=['original_timestamp'], low_memory=False)
    print("Anonymized data loaded successfully.")
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please ensure your anonymized CSV files are in the 'data' directory.")

# Display initial data shapes and head
print(f"Calls data shape: {calls_df.shape}")
print(f"Census data shape: {census_df.shape}")
calls_df.head()

In [ ]:
# 2. Process and Categorize Call Data
print("\nStep 2: Processing and categorizing call data...")

# --- Filter out 'Unknown' hospital data ---
initial_rows = len(calls_df)
print(f"Rows before filtering 'Unknown' hospital: {initial_rows}")
calls_df = calls_df[calls_df['hospital_name'] != 'Unknown'].copy()
print(f"Removed {initial_rows - len(calls_df)} rows from 'Unknown' hospital.")
print(f"Rows remaining: {len(calls_df)}")


# Select necessary columns for the model and drop rows with missing essential data
calls_df = calls_df[['original_timestamp', 'organization_id', 'call_type']].copy()
calls_df.dropna(subset=['original_timestamp', 'organization_id', 'call_type'], inplace=True)
calls_df['call_type'] = calls_df['call_type'].astype(str)

# categorize_call function (based on 128 unique types)
def categorize_call(call_type):
    """Maps a specific call_type string to a broader category based on actual data."""
    call_type_lower = call_type.lower()
    
    # Category 1: Clinical
    clinical_keywords = [
        'pain', 'nausea', 'medication', 'glucose', 'sugar', 'insulin',
        'diabetic', 'vomiting', 'beeping', 'dressing', 'ekg', 'tums',
        'purewick', 'chg bath', 'charge nurse'
    ]
    if any(keyword in call_type_lower for keyword in clinical_keywords):
        return 'Clinical'
        
    # Category 2: Mobility
    mobility_keywords = [
        'restroom', 'walk', 'bed', 'reposition', 'commode', 'bedpan',
        'urinal', 'shower', 'bath', 'bathroom'
    ]
    if any(keyword in call_type_lower for keyword in mobility_keywords):
        return 'Mobility'
        
    # Category 3: Basic Need / Food & Drink
    basic_need_keywords = [
        'thirsty', 'hungry', 'water', 'meal', 'food', 'snack', 'juice',
        'coffee', 'tea', 'milk', 'soda', 'ice', 'jello', 'pudding',
        'yogurt', 'cracker', 'cheese', 'ensure', 'fruit', 'sauce',
        'peanut butter', 'cheerios', 'popsicle', 'tray'
    ]
    if any(keyword in call_type_lower for keyword in basic_need_keywords):
        return 'Basic Need'

    # Category 4: Housekeeping & Room Environment
    housekeeping_keywords = [
        'housekeeping', 'linen', 'towel', 'soap', 'floor', 'trash', 'paper',
        'sanitizer', 'clean-up', 'pillowcase', 'sheets', 'blanket',
        'gown', 'lights on / off', 'too hot', 'too cold', 'heat pack'
    ]
    if any(keyword in call_type_lower for keyword in housekeeping_keywords):
        return 'Housekeeping'

        
    # Category 5: Other / Administrative
    else:
        return 'Other'

# Apply the function to create a new 'call_category' column
calls_df['call_category'] = calls_df['call_type'].apply(categorize_call)
print("Call categories created.")
calls_df.head()

In [ ]:
# 3. Aggregate, Pivot, and Merge
print("\nStep 3: Aggregating calls and merging with census data...")

hourly_calls = calls_df.copy()
hourly_calls['timestamp_hour'] = hourly_calls['original_timestamp'].dt.floor('h')

# Group by organization, the new hour column, and the category
hourly_calls_pivoted = hourly_calls.groupby(['organization_id', 'timestamp_hour', 'call_category']) \
                                   .size() \
                                   .unstack(fill_value=0) \
                                   .reset_index()

# Prepare census data for merging
census_cols = ['original_timestamp', 'organization_id', 'rooms_with_patients']
census_df_clean = census_df[census_cols].copy()
census_df_clean.dropna(inplace=True)
census_df_clean.drop_duplicates(subset=['organization_id', 'original_timestamp'], keep='last', inplace=True)

print("Preparing data for merge...")

# Ensure consistent data types
hourly_calls_pivoted['organization_id'] = hourly_calls_pivoted['organization_id'].astype(int)
census_df_clean['organization_id'] = census_df_clean['organization_id'].astype(int)

# Round timestamps to nearest hour for exact matching
hourly_calls_pivoted['timestamp_rounded'] = hourly_calls_pivoted['timestamp_hour'].dt.round('h')
census_df_clean['timestamp_rounded'] = census_df_clean['original_timestamp'].dt.round('h')

# Perform the merge
final_df = pd.merge(
    hourly_calls_pivoted,
    census_df_clean[['organization_id', 'timestamp_rounded', 'rooms_with_patients']],
    on=['organization_id', 'timestamp_rounded'],
    how='left'
)

# Clean up the merged dataframe
final_df.drop(columns=['timestamp_rounded'], inplace=True)
final_df.dropna(subset=['rooms_with_patients'], inplace=True)

# Filter out records where census is zero
print(f"Data merged successfully. Shape before cleaning: {final_df.shape}")
initial_rows = len(final_df)
final_df = final_df[final_df['rooms_with_patients'] > 0].copy()

print(f"Final dataframe shape after cleaning: {final_df.shape}")
final_df.head()

In [ ]:
# Cell 4: Outlier Removal and Visualizations (UPDATED)
print("\nStep 4: Removing outliers and generating visualizations...")

# Define the final list of categories
call_categories = ['Clinical', 'Mobility', 'Basic Need', 'Housekeeping', 'Other']

# Ensure all expected category columns exist in the DataFrame, filling with 0 if not
for col in call_categories:
    if col not in final_df.columns:
        final_df[col] = 0

# Create a 'total_calls' column for analysis
final_df['total_calls'] = final_df[call_categories].sum(axis=1)

# --- NEW Outlier Removal Step ---
# Remove any single hour where the total number of calls on a unit exceeded 30
print(f"Shape before hourly outlier removal: {final_df.shape}")
initial_rows = len(final_df)

final_df = final_df[final_df['total_calls'] <= 30].copy()

rows_removed = initial_rows - len(final_df)
print(f"Removed {rows_removed} rows where total hourly calls were > 30.")
print(f"Shape after hourly outlier removal: {final_df.shape}")


# Continue with visualizations using the cleaned final_df
final_df['hour_of_day'] = final_df['timestamp_hour'].dt.hour
final_df['day_of_week'] = final_df['timestamp_hour'].dt.dayofweek # Monday=0, Sunday=6


# Visualization 1: Average Calls by Category per Day of the Week
print("\nGenerated Visualization 1: Average Calls by Category per Day of the Week.")

daily_category_totals = final_df.groupby('day_of_week')[call_categories].sum()
unique_day_counts = final_df.groupby('day_of_week')['timestamp_hour'].apply(lambda x: x.dt.date.nunique())
daily_category_avg = daily_category_totals.div(unique_day_counts, axis=0)
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_category_avg.index = daily_category_avg.index.map(lambda x: day_names[x])
daily_category_avg.plot(kind='bar', stacked=True, figsize=(14, 7),
                        colormap='viridis')

plt.title('Average Calls by Category per Day of the Week', fontsize=16)
plt.xlabel('Day of the Week', fontsize=12)
plt.ylabel('Average Number of Calls', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Call Category')
plt.tight_layout()
plt.show()


# Visualization 2: Average Calls by Hour of Day
print("Visualization 2: Average Calls by Hour.")
hourly_avg = final_df.groupby('hour_of_day')['total_calls'].mean()
plt.figure(figsize=(12, 6))
hourly_avg.plot(kind='bar', color='skyblue')
plt.title('Average Call Volume by Hour of the Day', fontsize=16)
plt.xlabel('Hour of Day (24-hour format)', fontsize=12)
plt.ylabel('Average Number of Calls', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


# Visualization 3: Relationship between Patient Census and Call Volume
print("Visualization 3: Call Volume vs. Patient Census.")
plt.figure(figsize=(10, 6))
final_df['rooms_with_patients'] = pd.to_numeric(final_df['rooms_with_patients'])
sns.scatterplot(data=final_df, x='rooms_with_patients', y='total_calls', alpha=0.3)
plt.title('Hourly Call Volume vs. Number of Patients', fontsize=16)
plt.xlabel('Number of Rooms with Patients (Census)', fontsize=12)
plt.ylabel('Total Calls in that Hour', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Cell 5
print("\nStep 5: Filtering data to the last 365 days for model training...")

# Find the most recent date in the dataset
latest_date = final_df['timestamp_hour'].max()
start_date = latest_date - pd.Timedelta(days=365)

print(f"Full data range: {final_df['timestamp_hour'].min()} to {latest_date}")
print(f"Filtering to training data from {start_date} onwards.")

model_df = final_df[final_df['timestamp_hour'] >= start_date].copy()

print(f"Shape of full dataframe: {final_df.shape}")
print(f"Shape of model dataframe (last year): {model_df.shape}")

In [ ]:
# Cell 6
print("\nStep 6: Engineering features for the model...")

# Create simple time-based features
model_df['hour_of_day'] = model_df['timestamp_hour'].dt.hour
model_df['day_of_week'] = model_df['timestamp_hour'].dt.dayofweek # Monday=0, Sunday=6

# --- CRITICAL: One-Hot Encode the organization_id ---
# This allows the model to learn unit-specific patterns
org_dummies = pd.get_dummies(model_df['organization_id'], prefix='organization_id')
model_df = pd.concat([model_df, org_dummies], axis=1)

# Define our features (X) and targets (y)
# Start with the base features
features = [
    'rooms_with_patients',
    'hour_of_day',
    'day_of_week'
]
# Add the new one-hot encoded organization columns to the feature list
features.extend(org_dummies.columns)


# Use the final list of categories as our targets
targets = ['Clinical', 'Mobility', 'Basic Need', 'Housekeeping', 'Other']

# Ensure the target columns exist and are numeric, fill NaNs just in case
for col in targets:
    if col not in model_df.columns:
        model_df[col] = 0
    model_df[col] = pd.to_numeric(model_df[col], errors='coerce').fillna(0)

X = model_df[features]
y = model_df[targets]

print("Features (X) and Targets (y) created.")
print(f"Total number of features: {len(features)}")
X.head()

In [ ]:
# Cell 7
print("\nStep 7: Training and evaluating the model...")

# Sort by time before splitting to ensure chronological order 
model_df.sort_values('timestamp_hour', inplace=True)
X = model_df[features]
y = model_df[targets]

# Split data chronologically (80% train, 20% test)
split_index = int(len(X) * 0.8)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"Training set size: {len(X_train)} rows")
print(f"Testing set size: {len(X_test)} rows")

# Initialize and train the Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, min_samples_leaf=5)
model.fit(X_train, y_train)
print("Model training complete.")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model and print accuracy metrics
print("\n--- Model Evaluation (Accuracy Metrics) ---")
y_pred_df = pd.DataFrame(y_pred, columns=targets)

for category in targets:
    mae = mean_absolute_error(y_test[category], y_pred_df[category])
    r2 = r2_score(y_test[category], y_pred_df[category])
    print(f"\nCategory: {category}")
    print(f"  Mean Absolute Error (MAE): {mae:.2f} calls")
    print(f"  R-squared (R²): {r2:.2f}")

# Overall performance for total calls
total_mae = mean_absolute_error(y_test.sum(axis=1), y_pred.sum(axis=1))
print(f"\nOverall MAE for Total Calls: {total_mae:.2f} calls")

In [ ]:
# Cell 8
print("\nStep 8: Saving model and feature columns...")

# Define the paths to save the files in the new 'models' directory
MODELS_DIR = '../models'
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

model_path = os.path.join(MODELS_DIR, 'call_forecasting_model.pkl')
columns_path = os.path.join(MODELS_DIR, 'model_feature_columns.pkl')

# Save the trained model
joblib.dump(model, model_path)

# Save the list of feature columns
joblib.dump(features, columns_path)

print(f"Model saved to: {model_path}")
print(f"Feature columns saved to: {columns_path}")